In [10]:
import requests
import time 
import random
import json

HEADERS = {
    "Cookie": "",
    "User-Agent": "",
    'Content-Type': 'application/json'
}

URL = 'https://rmfyalk.court.gov.cn/cpws_al_api/api/cpwsAl/search'
URL2 = 'https://rmfyalk.court.gov.cn/cpws_al_api/api/cpwsAl/content'

In [ ]:
all_data = []

for page in range(1,15):  # 循环30次，从第1页到第30页
    page_res = []
    payload = {
        "page": page,
        "size": 50,
        "lib": "qb",
        "searchParams": {
            "userSearchType": 1,
            "isAdvSearch": "0",
            "selectValue": "qw",
            "lib": "cpwsAl_qb",
            "sort_field": "",
            "case_sort_id_cpwsAl": "01"
        }
    }
    
    # 一级页面
    response = requests.post(url=URL, json=payload, headers=HEADERS)
    
    if response.status_code != 200:
        print(f"Failed to retrieve data on page {page}, status code: {response.status_code}")
        break
    print(f'=====[success craw page]: {page}=====')
    
    # 二级页面
    for item in json.loads(response.text)['data']['datas']:
        
        payload = {
            "gid":item['id']
        }
        
        response = requests.get(url=URL2, json=payload, headers=HEADERS)
        if response.status_code != 200:
            print(f"[error][page:{page}][id:{item['id']}], status code: {response.status_code}")
            break
        print(f"[success craw content]: page{page}[id:{item['id']}], status code: {response.status_code}")
        
        json_response = json.loads(response.text)['data']['data']
        res = {
            "案例": json_response['cpws_al_title'],
            "关键词":json_response['cpws_al_keyword'],
            "裁判要点": json_response['cpws_al_cpyz'],
            "基本案情": json_response.get('cpws_jbaq',''),
            "裁判结果":json_response.get('cpws_cpjg',''),
            "裁判理由": json_response.get('cpws_cply',''),
        }
        page_res.append(res)
        time.sleep(random.randint(2, 5))
        

    # 将当前数据写入
    with open(f"d:/desktop/craw/craw_page_{page}.json", "w") as file:
        json_res = json.dumps(page_res, ensure_ascii=False, indent=4)
        file.write(json_res)
    

    